# Plotting Tools

In [86]:
import os
import csv

import numpy as np
from scipy.io import loadmat, savemat
import matplotlib.pyplot as plt
import seaborn as sns 
# sns.set() #sns.set_style('dark'
sns.set(context='paper', style='darkgrid', font_scale=1.8)
import scipy.stats as stats


### Basic plotting example

In [ ]:
env = 'cartpole'
min_num_trials = 10
log_dir = './outputs/mass_runs/{}'.format(env)     # Directory specified in script, not including date+time

COLORS = ['#2ca02c',  '#9467bd', '#e377c2', '#17becf', '#d62728', '#1f77b4']

TASK_HORIZONS = {
    'cartpole': 200,
    'halfcheetah': 1000,
    'pusher': 150,
    'reacher': 150,
    'ant': 1000
}

LINEWIDTH=2.5
MARKERSIZE=14

TASK_HORIZON = TASK_HORIZONS[env]

plt.figure(figsize=(10, 6))




num_exp = float('inf')
for i, exp in enumerate(sorted(os.listdir(log_dir))):
    returns = []
    for subdir in os.listdir(os.path.join(log_dir, exp)):
        print(subdir)
        data = loadmat(os.path.join(log_dir, exp, subdir, "logs.mat"))
        if data["returns"].shape[1] >= min_num_trials:
            print(exp, subdir, data['returns'][0][min_num_trials-1])
            returns.append(data["returns"][0][:min_num_trials])

    returns = np.array(returns)
    returns = np.maximum.accumulate(returns, axis=-1)
    num_exp = returns.shape[0]# min(num_exp, returns.shape[0])
    if num_exp == 0:
        continue
        
    print(exp, num_exp)
    mean = np.mean(returns, axis=0)
    err = stats.sem(returns, axis=0)
    zorder = 5
    if exp.endswith('-calibrate'):
        zorder=10
    if exp == 'PE-TS1':
#         continue
        plt.plot(TASK_HORIZON*np.arange(1, min_num_trials + 1)[::10], mean[::10], label=exp, color=COLORS[3], marker='o', markersize=MARKERSIZE, linewidth=LINEWIDTH)
    else:
        plt.plot(TASK_HORIZON*np.arange(1, min_num_trials + 1), mean, label=exp, color=COLORS[i], linewidth=LINEWIDTH, zorder=zorder)
        plt.fill_between(TASK_HORIZON*np.arange(1, min_num_trials + 1), mean - err, mean + err, alpha=0.25, color=COLORS[i])

    upp = np.max(returns, axis=0)
    low = np.min(returns, axis=0)        
    plt.fill_between(TASK_HORIZON*np.arange(1, min_num_trials + 1), low, upp, alpha=0.10, color=COLORS[i])


plt.legend(loc='best')

plt.title("{}".format(env[0].upper() + env[1:]), fontsize=25)
plt.xlabel("Number of Timesteps")
plt.ylabel("Reward")

plt.tight_layout()


plt.show()
# plt.savefig('final_outputs/performance_{}_wpets.png'.format(env), bbox_inches='tight')